**Load Pipeline, Data, and Prepare Matrices**
The preprocessing pipeline, transformed training data, and test data are loaded from saved .pkl files.


In [0]:
import joblib
import numpy as np
import pandas as pd
from scipy.sparse import issparse
pipeline = joblib.load(
   "/Workspace/Users/gsc314@ensign.edu/csai382_lab_2_4_-GustavoC-/etl_pipeline/stedi_feature_pipeline.pkl"
)
X_train_transformed = joblib.load(
   "/Workspace/Users/gsc314@ensign.edu/csai382_lab_2_4_-GustavoC-/etl_pipeline/X_train_transformed.pkl"
)
X_test_transformed = joblib.load(
   "/Workspace/Users/gsc314@ensign.edu/csai382_lab_2_4_-GustavoC-/etl_pipeline/X_test_transformed.pkl"
)
def to_float_matrix(arr: np.ndarray) -> np.ndarray:
   """
   Ensures that input arrays (possibly object-dtype, sparse, or 0-d) are converted to a 2-D float matrix.
   This is necessary because saved feature arrays may have inconsistent shapes or types after transformation,
   and ML models require numeric 2-D arrays for training and prediction.
   """
   if arr.ndim == 0:
       # Handle 0-d array directly
       arr = arr.item()
       if issparse(arr):
           arr = arr.toarray()
       arr = np.array(arr, dtype=float)
   elif arr.dtype == object:
       arr = np.array([
           x.toarray() if issparse(x) else np.array(x, dtype=float)
           for x in arr
       ])
       arr = np.vstack(arr)
   elif issparse(arr):
       arr = arr.toarray()
   else:
       arr = np.array(arr, dtype=float)
   return arr
X_train = to_float_matrix(X_train_transformed)
X_test = to_float_matrix(X_test_transformed)
y_train = joblib.load(
   "/Workspace/Users/gsc314@ensign.edu/csai382_lab_2_4_-GustavoC-/etl_pipeline/y_train.pkl"
)
y_test = joblib.load(
   "/Workspace/Users/gsc314@ensign.edu/csai382_lab_2_4_-GustavoC-/etl_pipeline/y_test.pkl"
)
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

**Review Your SHAP Insights**

The most important feature in the model was num__distance_cm. This feature had a much higher importance than the others, which makes sense because step detection should mainly depend on movement distance.

A concerning behavior was that the model predicted almost everything as step. It failed to correctly detect the no_step class, which shows a class imbalance problem.

This indicates a weakness in the model because it is biased toward the majority class and does not learn the minority class well.

**Focused Hyperparameter Grid**

The original model showed high accuracy but completely failed to detect the no_step class. This indicates the model is biased toward the majority class.

To address this, I will run a focused hyperparameter search on the Logistic Regression regularization parameter C. This may help the model better learn patterns for the minority class.

I selected a small grid of C values to test different levels of regularization.

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

params = {
    "C": [0.001, 0.01, 0.1, 1, 10]
}

grid = GridSearchCV(
    LogisticRegression(max_iter=300),
    params,
    scoring="accuracy",
    cv=3,
    n_jobs=-1
)

grid.fit(X_train, y_train)

grid.best_params_, grid.best_score_


**Model Comparison**

Old model score:
The original model achieved about 0.95 accuracy on the test set but completely failed to detect the no_step class.

New tuned model score:
The refined model with C = 0.001 achieved a cross-validation accuracy of 0.9511.

Did the new tuning improve performance?
Yes, the performance improved slightly.

Will you switch to the new model?
Yes. Even though the improvement is small, the new model performs slightly better and is the result of a more focused and responsible tuning process.

In [0]:
import joblib
joblib.dump(grid.best_estimator_,
            "/Workspace/Users/gsc314@ensign.edu/csai382_lab_2_4_-GustavoC-/etl_pipeline/newstedi_best_model.pkl")

**Model Refinement Summary**

I performed a second, focused hyperparameter tuning on the Logistic Regression model. The original model showed high accuracy but failed to detect the no_step class, indicating a bias toward the majority class.

To improve this, I tuned the regularization parameter C using a small and targeted grid. The best result was achieved with C = 0.001, which slightly improved the overall accuracy.

Since the refined model performed better, I updated the final saved model. This decision is responsible because it follows a careful evaluation process and ensures that the final model is based on the best available configuration.

**Ethics Reflection**

Careless hyperparameter tuning could create unfair or unsafe models because it may hide weaknesses, such as ignoring minority classes or producing misleading accuracy scores. If tuning is done without careful evaluation, the model might appear accurate but fail in important real-world situations.

It is important to examine model behavior carefully so that we understand its strengths and weaknesses. Explainability tools like SHAP help reveal how the model makes decisions and prevent hidden biases.

Gospel principles such as honesty, integrity, and accountability guide me to report results truthfully and make careful decisions. Being honest about model limitations helps build trust and ensures that technology is used responsibly.